<a href="https://colab.research.google.com/github/Salahudin77/thesis/blob/main/python/medium_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 🟡 Medium Model: Small CNN (2 hidden layers)

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import time, psutil, os

# Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train_cnn = x_train.reshape((-1, 28, 28, 1))
x_test_cnn = x_test.reshape((-1, 28, 28, 1))

# Model
model_medium = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model_medium.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Train & Benchmark
start_time = time.time()
model_medium.fit(x_train_cnn, y_train, epochs=5, batch_size=64, validation_split=0.1)
end_time = time.time()

test_loss, test_acc = model_medium.evaluate(x_test_cnn, y_test)
mem_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)


print("Medium Model Results:")
print(f" Training Time: {end_time - start_time:.2f}s")
print(f" Accuracy: {test_acc:.4f}, Loss: {test_loss:.4f}")
print(f" Memory Usage: {mem_usage:.2f} MB")

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 53s 59ms/step - accuracy: 0.8676 - loss: 0.4410 - val_accuracy: 0.9817 - val_loss: 0.0599
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 81s 58ms/step - accuracy: 0.9814 - loss: 0.0592 - val_accuracy: 0.9858 - val_loss: 0.0479
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 82s 58ms/step - accuracy: 0.9878 - loss: 0.0383 - val_accuracy: 0.9892 - val_loss: 0.0403
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 53s 63ms/step - accuracy: 0.9910 - loss: 0.0286 - val_accuracy: 0.9892 - val_loss: 0.0445
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 76s 56ms/step - accuracy: 0.9918 - loss: 0.0249 - val_accuracy: 0.9902 - val_loss: 0.0362
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9859 - loss: 0.0404
Medium Model Results:
 Training Time: 379.67s
 Accuracy: 0.9884, Loss: 0.0335
 Memory Usage: 1370.79 MB
